# 信頼できるAIエージェントの構築（コードサンプル）

このノートブックは、信頼できるAIエージェントを構築するための実装例を日本語でまとめたものです。
- 安全性: セーフティフィルタ、サンドボックス、権限最小化
- セキュリティ/プライバシー: 秘密情報の取扱い、PIIマスキング
- 再現性/可観測性: ロギング、メトリクス、テスト、評価
- Human-in-the-Loop: ユーザー承認・中断・フィードバックの導入

## このノートブックについて（日本語版）

このノートブックは、英語版の `06-building-trustworthy-agents/code_samples/06-system-message-framework.ipynb` をもとに、日本語で分かりやすく再構成したものです。

英語版に対応する主要セクション:
- システムメッセージフレームワークの解説
- メタシステムメッセージの例
- 基本プロンプトの例
- 最適化されたシステムメッセージの例

日本語版の拡張（実務向けの足場を追加）:
- Human-in-the-Loop の概要
- 実行準備（依存関係のインストール、環境変数の安全な読み込み）
- 可観測性（構造化ロギング、トレースID）
- 入出力のバリデーション（Pydantic）と厳格なJSONパース
- セーフティ（NGワードとPIIの検出・マスキング）

注: 英語版の意図を保ちつつ、学習者がそのまま転用できる実装サンプルを補っています。

## システムメッセージフレームワークの解説（日本語版）

- 目的: エージェントの役割・責務・制約・口調を明確化し、再利用可能なテンプレートとして設計する
- 手順: メタ → 基本 → 最適化（LLMで整形・補強）
- ポイント: 同一フレームで複数エージェント（検索/計画/実行）へ展開可能、差分は変数化して注入

### メタシステムメッセージ例（日本語）
```plaintext
あなたはAIエージェント用アシスタントのシステムプロンプト作成に精通した専門家です。
会社名、役割、責務などの情報が与えられます。これらを用いてシステムプロンプトを作成してください。
システムプロンプトはできるだけ具体的に記述し、LLM を用いるシステムが
AI アシスタントの役割と責務を正確に理解できるよう、明確な構造を与えてください。
```

### 基本プロンプト例（日本語）
```plaintext
あなたは Contoso Travel の旅行代理店で、顧客の航空券予約が得意な AI アシスタントです。
顧客を支援するために次のタスクを実行できます：利用可能なフライトの検索、フライトの予約、
座席やフライト時間の希望の確認、予約済みフライトのキャンセル、フライトの遅延や欠航の通知。
```

### 最適化されたシステムメッセージ（最後のセルを実行すると得られる日本語サンプル）
以下が適切なシステムプロンプトの例です：

**システムプロンプト:**

あなたは「Contoso Travel」の旅行代理店の担当者です。あなたの主な役割は、航空券の予約と関連するサービスを提供することです。以下の責務を慎重かつ丁寧に遂行してください：

### **役割と責務**
1. **航空券予約:**
   - 顧客の希望する旅行日時、出発地、目的地、希望する航空会社や座席クラスを基に最適な航空券を検索し提案します。
   - 航空券の料金、利用条件（キャンセルポリシー、変更ポリシー等）を明確に説明します。
   - 必要に応じて複数のオプションを提示し、顧客が選びやすいようにサポートします。

2. **顧客情報の確認:**
   - 必要な顧客情報（フルネーム、パスポート番号、連絡先、特別な要望など）を正確に収集します。
   - 提供された情報の正確性を確認し、誤りを防ぐために慎重に取り扱います。

3. **トランザクション処理:**
   - 利用可能な支払いオプションを案内し、予約手続きを確定します。
   - 予約完了後、航空券の確認番号や電子チケットを迅速に提供します。

4. **追加サービスの案内:**
   - バゲージオプション、特別サービス（特別食事、車椅子利用など）、旅行保険などの追加オプションを適宜提案します。

5. **アフターサポート:**
   - 航空券の変更、キャンセル、払い戻しに関する問い合わせを適切に対応します。
   - フライト情報（遅延やキャンセルなど）の更新を提供し、必要に応じて代替案を提案します。

### **対応時の行動指針**
- 丁寧かつプロフェッショナルなコミュニケーションを心がけます。
- 顧客の希望を最大限理解し、迅速かつ正確に対応します。
- 必要に応じて専門的な情報を分かりやすい形式で説明します。

### **制限事項**
- 個人情報は厳格に保護し、許可された目的以外で利用しません。
- 法律や規定に触れるようなアドバイスや手続きは行いません。

あなたの目的は、顧客の旅行体験をスムーズで快適にするために、最良の航空券予約サービスを提供することです。

--- 

これにより、AIアシスタントは与えられた役割と責務を正確に理解し、適切なサービスを提供する準備ができます。

## Human-in-the-Loop（人間介入型）の解説

- 仕組み: ユーザーが実行中のエージェントの出力に対して承認/中断/修正依頼を行えるワークフロー。
- 利点: リスクの早期検出、品質の担保、説明責任の確保。
- 代表シナリオ: 機密操作の確認、費用の伴う処理の承認、重要文書の最終チェック。

In [1]:
# 必要ライブラリのインストールとNotebook設定
%pip install pydantic python-dotenv tenacity httpx pytest rich tqdm openai azure-ai-inference --quiet

# Jupyterの便利設定
%load_ext autoreload
%autoreload 2

# 乱数シード固定（再現性）
import os, random
import numpy as np
random.seed(42)
os.environ.setdefault("PYTHONHASHSEED", "42")
np.random.seed(42)


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [2]:
# 環境変数・シークレットの安全な読み込み
from dotenv import load_dotenv
load_dotenv()

import os

def mask(value: str, show: int = 3) -> str:
    if not value:
        return "(未設定)"
    if len(value) <= show * 2:
        return value[0:1] + "***" + value[-1:]
    return value[:show] + "***" + value[-show:]

# GitHub Models向けの環境変数を整備
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
if GITHUB_TOKEN and not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = GITHUB_TOKEN

if not os.getenv("OPENAI_BASE_URL"):
    os.environ["OPENAI_BASE_URL"] = "https://models.inference.ai.azure.com/"

print("OPENAI_API_KEY:", mask(os.getenv("OPENAI_API_KEY")))
print("OPENAI_BASE_URL:", os.getenv("OPENAI_BASE_URL"))

if not os.getenv("OPENAI_API_KEY"):
    print("⚠️ APIキーが設定されていません。.env に GITHUB_TOKEN を設定してください。")

OPENAI_API_KEY: git***RmQ
OPENAI_BASE_URL: https://models.inference.ai.azure.com/


In [3]:
# 構造化ロギングとトレースID
import logging, json, time
from contextvars import ContextVar

correlation_id: ContextVar[str] = ContextVar("correlation_id", default="-")

class JsonLikeFormatter(logging.Formatter):
    def format(self, record: logging.LogRecord) -> str:
        payload = {
            "level": record.levelname,
            "time": int(record.created * 1000),
            "logger": record.name,
            "message": record.getMessage(),
            "corr": correlation_id.get(),
        }
        return json.dumps(payload, ensure_ascii=False)

root = logging.getLogger()
for h in list(root.handlers):
    root.removeHandler(h)
handler = logging.StreamHandler()
handler.setFormatter(JsonLikeFormatter())
root.addHandler(handler)
root.setLevel(logging.INFO)

logger = logging.getLogger("trustworthy")
logger.info("ロギング初期化が完了しました")

{"level": "INFO", "time": 1755847714360, "logger": "trustworthy", "message": "ロギング初期化が完了しました", "corr": "-"}


In [4]:
# セル10の概要:
# - ユーザー入力スキーマ(UserInput)をPydanticで定義し、入力の型/制約を検証します。
# - goalは5文字以上かつNGワード(破壊/違法/不正)を拒否。tools_allowedは許可リストで制約します。
# - デモとしてサンプルを検証し、成功/失敗をロギングします。

# 入力スキーマ定義とバリデーション（Pydantic）
from pydantic import BaseModel, Field, field_validator
from typing import List, Literal
import re

class UserInput(BaseModel):
    goal: str = Field(min_length=5, description="ユーザーの最終目標")
    constraints: List[str] = Field(default_factory=list, description="制約条件のリスト")
    tools_allowed: List[Literal["calculator", "search", "none"]] = Field(default_factory=list)

    @field_validator("goal")
    @classmethod
    def goal_no_forbidden(cls, v: str) -> str:
        if re.search(r"(破壊|違法|不正)", v):
            raise ValueError("禁止語が含まれています。別の表現にしてください。")
        return v

try:
    sample = UserInput(goal="旅行計画を最適化する", constraints=["予算10万円"], tools_allowed=["calculator"])
    logger.info(f"入力検証OK: {sample}")
except Exception as e:
    logger.error(f"入力検証エラー: {e}")

{"level": "INFO", "time": 1755847718552, "logger": "trustworthy", "message": "入力検証OK: goal='旅行計画を最適化する' constraints=['予算10万円'] tools_allowed=['calculator']", "corr": "-"}


In [5]:
# セル11の概要:
# - LLM出力をPlanOutput/StepPlanのスキーマで厳格に検証し、```json フェンス内のみを抽出してパースします。
# - 失敗時はJsonFormatError/ValidationErrorで検出し、tenacityで最大3回リトライ（指数バックオフ）。
# - 主に防げる攻撃/不具合:
#   * 出力インジェクション/プロンプト汚染（JSON外の命令や長文の混入）
#   * パラメータ汚染・スキーマスミuggling（余計なキー/型不一致/未知値）
#   * コード/コマンド注入（JSONだけをデータとして扱い実行をしない）
#   * XSS/Markdown注入（フェンス外のHTML/Markdownを無視）
#   * 文章＋JSON混在による誤動作、パース失敗の悪用

# 出力スキーマ定義と厳格なJSONパース + 自己修復リトライ
from pydantic import BaseModel, Field, ValidationError
from typing import Optional
import json
import re
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

class StepPlan(BaseModel):
    step: int = Field(ge=1)
    action: str
    rationale: str

class PlanOutput(BaseModel):
    success: bool
    steps: list[StepPlan]
    next_hint: Optional[str] = None

class JsonFormatError(Exception):
    pass

# モデル出力から厳格にJSONを抽出するユーティリティ
JSON_FENCE = re.compile(r"```json\s*(\{[\s\S]*?\})\s*```", re.IGNORECASE)

@retry(
    reraise=True,
    stop=stop_after_attempt(3),
    wait=wait_exponential(multiplier=0.5, min=0.5, max=2),
    retry=retry_if_exception_type((JsonFormatError, ValidationError)),
)
def parse_plan_strict(text: str) -> PlanOutput:
    """
    - ```json ... ``` フェンスの中身のみ許可
    - パース失敗 or スキーマ不一致はリトライ
    """
    m = JSON_FENCE.search(text)
    if not m:
        logger.warning("JSONフェンスが見つかりません。全体から最初の { ... } ブロックを探索します。")
        # 最低限のフォールバック（最初の{...}ブロック）
        brace_start = text.find("{")
        brace_end = text.rfind("}")
        if brace_start == -1 or brace_end == -1 or brace_end <= brace_start:
            raise JsonFormatError("JSONブロックが抽出できませんでした")
        raw = text[brace_start: brace_end + 1]
    else:
        raw = m.group(1)

    try:
        obj = json.loads(raw)
    except json.JSONDecodeError as je:
        raise JsonFormatError(f"JSONデコード失敗: {je}") from je

    try:
        return PlanOutput.model_validate(obj)
    except ValidationError as ve:
        raise ve

# デモ
bad = """
以下はプランです：
1) 下見
2) 予約
```json
{"success": true, "steps": [{"step": 1, "action": "search", "rationale": "安い便を探す"}]}
```
その他の説明
"""

try:
    parsed = parse_plan_strict(bad)
    logger.info(f"厳格パースOK: steps={len(parsed.steps)}")
except Exception as e:
    logger.error(f"厳格パース失敗: {e}")

{"level": "INFO", "time": 1755847721146, "logger": "trustworthy", "message": "厳格パースOK: steps=1", "corr": "-"}


In [6]:
# セル12の概要:
# - 安全フィルタでNGワード検知とPII（メール/電話/クレカ）検出・マスキングを行います。
# - DENYLIST/PII_PATTERNSでヒットを抽出し、ログに記録。redacted_textで機微情報を置換します。
# - run_safety_filter(text) が SafetyReport を返し、下流処理へ安全なテキストを渡せます。
# - デモとしてサンプル文を処理し、検出結果とマスク後のテキストを表示します。

# 安全フィルタ：NGワード検知とPII検出・マスキング
from pydantic import BaseModel
import re
from typing import Dict, List

# 簡易NGワードリスト（デモ用）
DENYLIST = [
    r"破壊", r"違法", r"不正", r"ハッキング", r"マルウェア"
]

# PII（個人情報）検出用の簡易パターン（実運用では専用ライブラリを推奨）
PII_PATTERNS: Dict[str, re.Pattern] = {
    "EMAIL": re.compile(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}", re.I),
    "phone": re.compile(r"\b(?:\+?\d{1,3}[-.\s]?)?(?:\d{2,4}[-.\s]?){2,4}\d{2,4}\b"),
    "CREDIT_CARD": re.compile(r"\b(?:\d[ -]*?){13,19}\b"),
}

class SafetyReport(BaseModel):
    denylist_hits: List[str]
    pii_hits: Dict[str, List[str]]
    redacted_text: str


def detect_denylist(text: str) -> List[str]:
    hits = []
    for pat in DENYLIST:
        if re.search(pat, text, re.I):
            hits.append(pat)
    return hits


def detect_pii(text: str) -> Dict[str, List[str]]:
    hits: Dict[str, List[str]] = {}
    for label, pat in PII_PATTERNS.items():
        found = pat.findall(text)
        if found:
            hits[label] = found if isinstance(found, list) else [found]
    return hits


def redact_pii(text: str) -> str:
    red = text
    for label, pat in PII_PATTERNS.items():
        red = pat.sub(f"[REDACTED:{label}]", red)
    return red


def run_safety_filter(text: str) -> SafetyReport:
    deny_hits = detect_denylist(text)
    pii = detect_pii(text)
    redacted = redact_pii(text)
    if deny_hits:
        logger.warning(f"NGワード検出: {deny_hits}")
    if any(pii.values()):
        logger.info(f"PII検出: {[k for k,v in pii.items() if v]}")
    return SafetyReport(denylist_hits=deny_hits, pii_hits=pii, redacted_text=redacted)

# デモ
sample_text = "お問い合わせは admin@example.com まで。ハッキングの方法を教えて。電話は+81-90-1234-5678"
report = run_safety_filter(sample_text)
logger.info(f"安全フィルタ結果: deny={report.denylist_hits}, pii_keys={list(report.pii_hits.keys())}")
print("Redacted:", report.redacted_text)

{"level": "WARNING", "time": 1755847726671, "logger": "trustworthy", "message": "NGワード検出: ['ハッキング']", "corr": "-"}
{"level": "INFO", "time": 1755847726673, "logger": "trustworthy", "message": "PII検出: ['EMAIL', 'phone']", "corr": "-"}
{"level": "INFO", "time": 1755847726674, "logger": "trustworthy", "message": "安全フィルタ結果: deny=['ハッキング'], pii_keys=['EMAIL', 'phone']", "corr": "-"}
{"level": "INFO", "time": 1755847726673, "logger": "trustworthy", "message": "PII検出: ['EMAIL', 'phone']", "corr": "-"}
{"level": "INFO", "time": 1755847726674, "logger": "trustworthy", "message": "安全フィルタ結果: deny=['ハッキング'], pii_keys=['EMAIL', 'phone']", "corr": "-"}


{"level": "WARNING", "time": 1755847726671, "logger": "trustworthy", "message": "NGワード検出: ['ハッキング']", "corr": "-"}
{"level": "INFO", "time": 1755847726673, "logger": "trustworthy", "message": "PII検出: ['EMAIL', 'phone']", "corr": "-"}
{"level": "INFO", "time": 1755847726674, "logger": "trustworthy", "message": "安全フィルタ結果: deny=['ハッキング'], pii_keys=['EMAIL', 'phone']", "corr": "-"}
{"level": "INFO", "time": 1755847726673, "logger": "trustworthy", "message": "PII検出: ['EMAIL', 'phone']", "corr": "-"}
{"level": "INFO", "time": 1755847726674, "logger": "trustworthy", "message": "安全フィルタ結果: deny=['ハッキング'], pii_keys=['EMAIL', 'phone']", "corr": "-"}


Redacted: お問い合わせは [REDACTED:EMAIL] まで。ハッキングの方法を教えて。電話は[REDACTED:phone]


In [ ]:
# セル14の概要:
# - Azure AI Inference SDK を用いて役割/会社名/責務から構造化されたシステムプロンプト案を生成する最小デモ。
# - 認証: GITHUB_TOKEN または OPENAI_API_KEY。環境変数でモデル/エンドポイント切替。
# - 出力: モデルが生成したシステムプロンプト（実行毎に変動）。

import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.getenv('GITHUB_TOKEN') or os.getenv('OPENAI_API_KEY')
if not token:
    raise RuntimeError('GITHUB_TOKEN もしくは OPENAI_API_KEY が未設定です。.env に GITHUB_TOKEN を設定してください。')
endpoint = os.getenv('OPENAI_BASE_URL', 'https://models.inference.ai.azure.com')
model_name = os.getenv('OPENAI_MODEL', 'gpt-4o')

client = ChatCompletionsClient(endpoint=endpoint, credential=AzureKeyCredential(token))

role = '旅行代理店の担当者'
company = 'Contoso Travel'
responsibility = '航空券の予約'

response = client.complete(
    messages=[
        SystemMessage(content=(
            'あなたはAIエージェントのシステムプロンプト設計に精通した専門家です。'
            '会社名・役割・責務などの情報が与えられます。これらを用いて、'
            'AIアシスタントの役割と責務をシステムが正しく理解できるよう、'
            'できるだけ具体的で、構造化されたシステムプロンプトを作成してください。'
        )),
        UserMessage(content=f'あなたは {company} の {role} で、{responsibility} を担当しています。'),
    ],
    model=model_name,
    temperature=1.0,
    max_tokens=1000,
    top_p=1.0,
)

generated = response.choices[0].message.content
print(generated)